**jumbo rolls are folded in**

**changeover between jumbos are folded in**

**Minimize knife changes folded in**

need to ..

* show when an order in the future has been fullfilled.

* look forward in schedule to optimize new/better layouts

* make layout table editable

* speed test algorithm

* show inventory created

need to 2.0

* pull in the put up from the schedule

* date order is complete to configure slitter schedule start

why is knife changes creating so much more inventory than late orders? - it's actually not, the schedule totals are the same, maybe it's worth looking at what 'extras' is tabulating in each case, it would be good if it were the same for both methods.probably extras isn't being updated in the last loops cycles of each

We have tried adding in the completed orders to "late orders" optimization. it is complex. I think at this point it will be better to wait for overhaul of that code from dataframe to dictionaries/arrays to make this change

program fixes:

* make knike and late algs match eachother (speed up too)
* 

# Simple Genetic Algorithm to Optimize Deckle Loss

In [1]:
import sys
sys.path.append("../")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import math
from random import shuffle, choice
import copy
from utils import *
from engine import *
from IPython.display import clear_output
import time
import datetime
import re

In [2]:
df = pd.read_excel('../data/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
                   sheet_name='Schedule')

# the input schedule appears as a data table on the dashboard
# this is used to select what to deckle
df_input_schedule = pd.read_excel('../data/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
                   sheet_name='Schedule')
df_input_schedule.insert(1, 'Technology', df_input_schedule['Description'].apply(lambda x: parse_description(x, 'tech')))
df_input_schedule.insert(2, 'Color', df_input_schedule['Description'].apply(lambda x: x.split(';')[1] if type(x) == str else None))
df_input_schedule.insert(3, 'Width', df_input_schedule['Description'].apply(lambda x: parse_description(x, 'width')))
df_input_schedule = df_input_schedule[[col for col in df_input_schedule.columns if 'Unnamed' not in str(col)]]
df_input_schedule['Total LM Order QTY'] = df_input_schedule['Total LM Order QTY'].round(1)

In [3]:
df_input_schedule.columns

Index(['SPM375014                               PROD. DATE',
                                               'Technology',
                                                    'Color',
                                                    'Width',
                                           'CYCLE / BUCKET',
                                            'Customer Name',
                                                  'Item No',
                                                      'WO#',
                                              'Description',
                                                'Treatment',
                                                       'KG',
                                         'Hours To Produce',
                                                  'Polymer',
                                                'Core Size',
                                                 'LM putup',
                                            'Roll Diameter',
                        

In [4]:
temp = df_input_schedule[['Customer Name', 'Technology', 'Color', 'Width', 'CYCLE / BUCKET',
                          'Description', 'Total LM Order QTY', 'LM putup',
                          'Scheduled Ship Date', 'Date order is complete']]
temp.loc[:,'Scheduled Ship Date'] = pd.to_datetime(temp['Scheduled Ship Date'], errors='coerce')
temp.loc[:,"Scheduled Ship Date"] = temp["Scheduled Ship Date"].dt.strftime("%Y-%m-%d")

temp.loc[:,"Date order is complete"] = temp["Date order is complete"].dt.round('T')


/home/wbeckner/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [5]:
temp.head()

,Customer Name,Technology,Color,Width,CYCLE / BUCKET,Description,Total LM Order QTY,LM putup,Scheduled Ship Date,Date order is complete
0,P & G,SAM,WHITE,208,CYCLE 1,SAM0130UN0208;WHITE;300,369822.5,17000.0,2020-07-21,2020-07-20 13:05:00
1,P & G,SAM,WHITE,170,CYCLE 1,SAM0130UN0170;WHITE;300,2262443.4,17000.0,2020-07-21,2020-07-20 16:01:00
2,P & G,SAM,WHITE,234,CYCLE 1,SAM0130UN0234;WHITE;300,2005259.7,17000.0,2020-07-22,2020-07-20 19:35:00
3,P & G,SAM,WHITE,234,CYCLE 1,SAM0130UN0234;WHITE;300,2136752.1,17000.0,2020-07-22,2020-07-20 23:23:00
4,P & G,SAM,WHITE,234,CYCLE 1,SAM0130UN0234;WHITE;300,2136752.1,17000.0,2020-07-22,2020-07-21 03:11:00


In [6]:
# temp = temp[temp['Scheduled Ship Date'].apply(lambda x: type(x) == int)]                          
temp_json = temp.to_json()

In [7]:
temp2 = pd.read_json(temp_json, 
                     convert_dates=['Scheduled Ship Date', 'Date order is complete'])

## to_datetime and to_json use default units ms and ns, respectively
temp2['Scheduled Ship Date'] = pd.to_datetime(temp2['Scheduled Ship Date'], 
                                              errors='coerce', unit='ms')

In [8]:
temp2["Scheduled Ship Date"] = temp2["Scheduled Ship Date"].dt.strftime("%Y-%m-%d")

In [9]:
for i, x in enumerate(temp2['Scheduled Ship Date']):
    if type(x) == pd._libs.tslibs.timestamps.Timestamp:
        temp2.loc[i, 'Scheduled Ship Date'] = datetime.datetime.strftime(x, '%Y-%m-%d')

In [10]:
temp2

,Customer Name,Technology,Color,Width,CYCLE / BUCKET,Description,Total LM Order QTY,LM putup,Scheduled Ship Date,Date order is complete
0,P & G,SAM,WHITE,208.0,CYCLE 1,SAM0130UN0208;WHITE;300,369822.5,17000.0,2020-07-21,2020-07-20 13:05:00
1,P & G,SAM,WHITE,170.0,CYCLE 1,SAM0130UN0170;WHITE;300,2262443.4,17000.0,2020-07-21,2020-07-20 16:01:00
10,P & G,SAM,WHITE,234.0,CYCLE 1,SAM0130UN0234;WHITE;300,2136752.1,17000.0,2020-07-23,2020-07-22 01:59:00
100,None,None,None,NaN,CYCLE 1,None,NaN,NaN,NaT,2020-08-06 15:51:00
101,CARDINAL HEALTH,SM,WHITE,575.0,CYCLE 1,SM10120UN0575;WHITE;300,1143043.5,17900.0,2020-08-10,2020-08-06 21:12:00
102,CARDINAL HEALTH,SM,WHITE,626.0,CYCLE 1,SM10120UN0626;WHITE;300,1142971.2,17900.0,2020-08-10,2020-08-07 03:02:00
103,CARDINAL HEALTH,SM,WHITE,575.0,CYCLE 1,SM10120UN0575;WHITE;300,1143043.5,17900.0,2020-08-10,2020-08-07 08:22:00
104,CARDINAL HEALTH,SM,WHITE,626.0,CYCLE 1,SM10120UN0626;WHITE;300,1142971.2,17900.0,2020-08-10,2020-08-07 14:12:00
105,CARDINAL HEALTH,SM,WHITE,575.0,CYCLE 1,SM10120UN0575;WHITE;300,1143043.5,17900.0,2020-08-10,2020-08-07 19:32:00
106,CARDINAL HEALTH,SM,WHITE,626.0,CYCLE 1,SM10120UN0626;WHITE;300,1142971.2,17900.0,2020-08-10,2020-08-08 01:22:00


In [13]:
temp2['LM putup'].values[0]

17000.0

In [10]:
### not currently used, grabs indicated information
customer_names = [i for i in df['Customer Name'].unique() if type(i) == str]
technologies = list(set([parse_description(i, 'tech') for i in df['Description'] 
                         if (type(i) == str)]))
technologies = [i for i in technologies if i]
colors = list(set([i[1] for i in list(df['Description']
                .str.split(';')) if type(i) == list]))
cycles = list(df['CYCLE / BUCKET'].unique())

In [11]:
customer = 'FIRST QUALITY PRODUCTS'
technology = 'SM' # Technology -> 
color = 'DENBLUE'
cycle = 'CYCLE 1'

customer = 'P & G'
technology = 'SAM'
color = 'WHITE'
cycle = 'CYCLE 1'

customer = 'AHP'
technology = 'SM' # Technology -> 
color = 'WHITE'
cycle = 'CYCLE 1'

df_filtered = df.loc[df['Customer Name'] == customer]
df_filtered = df_filtered.loc[df_filtered['Description'].str.contains(technology)]
df_filtered = df_filtered.loc[df_filtered['Description'].str.contains(color)] #CYAN, TEAL
df_filtered = df_filtered.loc[df_filtered['CYCLE / BUCKET'] == cycle]
df_filtered = df_filtered.loc[df_filtered['Total LM Order QTY'] > 0]
df_filtered.insert(0, 'Block', 1)
df_filtered = df_filtered.reset_index(drop=True)
df_filtered['Order Number'] = df_filtered.index + 1

df_filtered['Width'] = pd.DataFrame(list(pd.DataFrame(list(df_filtered['Description']
                                                  .str.split(';')))[0].str.split('UN0')))[1]
df_filtered.head(10)

,Block,SPM375014 PROD. DATE,CYCLE / BUCKET,Customer Name,Item No,WO#,Description,Treatment,KG,Hours To Produce,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,% reclaim,Amount Repro needed,Type of Repro,Order Number,Width
0,1,2020-07-29 04:38:41.042,CYCLE 1,AHP,1559695,NaN,"SM30135UN0160;WHITE;600;46"" OD",UNTREATED,2000.0,1.131222,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,1,160
1,1,2020-07-29 05:46:33.441,CYCLE 1,AHP,1559618,NaN,"SM30135UN0195;WHITE;600;46"" OD",UNTREATED,7500.0,4.242081,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,2,195
2,1,2020-07-29 10:01:04.934,CYCLE 1,AHP,1715784,NaN,"SM30135UN0205;WHITE;600;46"" OD",UNTREATED,3000.0,1.696833,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,3,205
3,1,2020-07-29 11:42:53.531,CYCLE 1,AHP,1716031,NaN,"SM30135UN0220;WHITE;600;46"" OD",UNTREATED,14000.0,7.918552,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,4,220
4,1,2020-07-29 19:38:00.318,CYCLE 1,AHP,1559618,NaN,"SM30135UN0195;WHITE;600;46"" OD",UNTREATED,4725.0,2.672511,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,5,195
5,1,2020-07-29 22:18:21.359,CYCLE 1,AHP,1715784,NaN,"SM30135UN0205;WHITE;600;46"" OD",UNTREATED,7425.0,4.199661,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,6,205
6,1,2020-07-30 02:30:20.137,CYCLE 1,AHP,1716031,NaN,"SM30135UN0220;WHITE;600;46"" OD",UNTREATED,3037.0,1.717760,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,7,220
7,1,2020-07-30 04:13:24.074,CYCLE 1,AHP,1559695,NaN,"SM30135UN0160;WHITE;600;46"" OD",UNTREATED,5400.0,3.054299,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,8,160
8,1,2020-07-30 07:16:39.549,CYCLE 1,AHP,1559618,NaN,"SM30135UN0195;WHITE;600;46"" OD",UNTREATED,8100.0,4.581448,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,9,195
9,1,2020-07-30 11:51:32.762,CYCLE 1,AHP,1715784,NaN,"SM30135UN0205;WHITE;600;46"" OD",UNTREATED,8100.0,4.581448,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,10,205


In [12]:
### Detects if there is a break in the rolls coming of the
### line

prev_index = df_filtered.index[0] - 1
block = 1
for index in df_filtered.index:
    if index != prev_index + 1:
        block += 1
    df_filtered.loc[index, 'Block'] = block
    prev_index = index

In [13]:
list(df_filtered.groupby('Width')['Total LM Order QTY'].sum().index.values.astype(int))

[160, 195, 205, 220, 235]

In [14]:
lm = list(df_filtered.groupby('Description')['Total LM Order QTY'].sum().values)
widths = list(df_filtered.groupby('Width')['Total LM Order QTY'].sum().index.values.astype(int))
doffs_in_jumbo = 6
B = 4160
L = doff_length = 17000 * doffs_in_jumbo # df['LM putup']

# neckin = [4, 4, 5, 7, 7, 7] # 158 missing in cycle 1, 4 mm knife in
neckins = []
for width in widths:
    if width < 170:
        neckin = 4
    elif width < 208:
        neckin = 5
    else:
        neckin = 7
    neckins.append(neckin)
w = list(np.array(widths) + np.array(neckins)) # the values used in the actual computation
q = [math.ceil(x/L) for x in lm] # jumbos needed per width, rounded up
s = BinPackingExample(w, q) # material orders (list of widths, 1 width = 1 jumbo)

print('The important variables', end='\n\n')
print('widths: {} (mm)'.format(widths))
print('neck in: {} (mm)'.format(neckins))
print('L (m): {}'.format(L))
print('undeckled jumbos needed: {}'.format(q))

The important variables

widths: [160, 195, 205, 220, 235] (mm)
neck in: [4, 5, 5, 7, 7] (mm)
L (m): 102000
undeckled jumbos needed: [54, 344, 196, 212, 17]


In [15]:
# check that s has proper amount of jumbos
from collections import Counter
Counter(s)

Counter({164: 54, 200: 344, 210: 196, 227: 212, 242: 17})

if we gave each order its own entire row we would produce a lot of wasted material

# Run Deckle Optimizer

In [18]:
sol, loss = find_optimum(s, B, widths, neckins, max_unique_products=4, 
                         loss_target=12, max_doff_layouts=10)

4.41215034965035


/home/wab665/anaconda3/envs/py37/lib/python3.8/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



In [20]:
pd.DataFrame(sol)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,242,242,242,242,242,242,242,242,242,242,...,242,242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,227,227,227,210,210,210,210,210,210,210,...,210,210,210.0,210.0,NaN,NaN,NaN,NaN,NaN,NaN
2,210,210,210,210,210,210,210,210,210,210,...,210,210,210.0,210.0,164.0,NaN,NaN,NaN,NaN,NaN
3,210,210,210,210,210,210,210,210,210,210,...,210,210,210.0,210.0,164.0,NaN,NaN,NaN,NaN,NaN
4,210,210,210,210,210,210,210,210,210,210,...,210,210,210.0,210.0,164.0,NaN,NaN,NaN,NaN,NaN
5,210,210,210,210,210,210,210,210,210,210,...,210,200,200.0,200.0,200.0,NaN,NaN,NaN,NaN,NaN
6,200,200,200,200,200,200,200,200,200,200,...,200,200,200.0,200.0,200.0,NaN,NaN,NaN,NaN,NaN
7,200,200,200,200,200,200,200,200,200,200,...,200,200,200.0,200.0,200.0,NaN,NaN,NaN,NaN,NaN
8,200,200,200,200,200,200,200,200,200,200,...,164,164,164.0,164.0,164.0,164.0,164.0,NaN,NaN,NaN
9,164,164,164,164,164,164,164,164,164,164,...,164,164,164.0,164.0,164.0,164.0,164.0,164.0,164.0,164.0


In [154]:
# def find_optimum(s,
#              B,
#              widths,
#              neckin,
             
loss_target = 12
max_doff_layouts = 20
max_unique_products = 6
gene_count = 5
doff_min = 1


losses = []
solutions = []
best_loss = 100
step = 0
start_time = time.time()

while True:
    step += 1
    shuffle(s)
    remain = [B] #initialize list of remaining bin spaces
    sol = [[]]
    binlim = np.inf

    for item in s:#sorted(s, reverse=True): # iter through products
        for j,free in enumerate(remain): #
            if (free >= item) and (len(sol[j]) < binlim): # if theres room
                remain[j] -= item
                sol[j].append(item)
                break
        else: # at this step we need to double the previous (or mult. by min. doff count)
            sol.append([item]) #starts new list in sol list
            remain.append(B-item) #append a new bin and subtract the width of the new item

    genes = []

    df = pd.DataFrame(sol)
    df = df.fillna(0)
    dff = pd.DataFrame(df.groupby(list(df.columns)).size()).\
        rename(columns={0: 'freq'}).reset_index()
    dff = dff[dff.columns[:-1]]
    dff["Loss"] = B - dff.sum(axis=1)
    dff = dff.loc[dff.replace(0, np.nan, inplace=False).nunique(axis=1) <= max_unique_products]
    dff = dff.loc[dff['Loss'] < 100]
    dff = dff.sort_values('Loss').head(gene_count).reset_index(drop=True)

    dff = dff[dff.columns[:-1]]
    dff = dff.fillna(0)

    for row in dff.index:
        gene = list(dff.iloc[row].values)
        gene = [i for i in gene if i != 0]
        genes.append(gene)

    order_remaining = copy.copy(s)
    sol2 = []
    gene_time = time.time() - start_time

    while True:
        backup_order_remaining = copy.copy(order_remaining)
        try:
            new_gene = list(choice(genes))
        except:
            break
        if new_gene in sol2:
            sol2.append(new_gene)
            for item in new_gene:
                if item in order_remaining:
                    order_remaining.remove(item)
                elif item == 0:
                    pass
                else: # new gene over produces item
                    order_remaining = backup_order_remaining
                    sol2.remove(new_gene)
                    genes.remove(new_gene)
                    break
        else:
            check_pass = True
            for mult in range(doff_min):

                for item in new_gene:
                    if item in order_remaining:
                        order_remaining.remove(item)
                    elif item == 0:
                        pass
                    else: # new gene over produces item

                        check_pass = False
                        break
                else:

                    continue  # only executed if the inner loop did NOT break
                if check_pass == False:  # only executed if the inner loop DID break
                    order_remaining = backup_order_remaining
                    genes.remove(new_gene)

                    break
            if check_pass == True:
                for mult in range(doff_min):
                    sol2.append(new_gene)

    chrome_time = time.time() - gene_time
    #     if step > iterations:
    #         break
    remain2 = [B] #initialize list of remaining bin spaces
#         print(remain2)
    sol3 = [[]]
    binlim = np.inf
    doff_min = 2
    for item in sorted(order_remaining, reverse=True): # iter through products
        for j,free in enumerate(remain2): #
            if (free >= item) and (len(sol3[j]) < binlim): # if theres room,
                remain2[j] -= item # and we haven't reach bimlim
                sol3[j].append(item)
                break
        else: # at this step we need to double the previous (or mult. by min. doff count)
            sol3.append([item]) #starts new list in sol list
            remain2.append(B-item) #append a new bin and subtract the width of the new item
            # subtract
    # loss = sum(remain2) / np.sum(np.sum(sol3)) * 100
    ffd_time = time.time() - chrome_time
    sol_tot = sol2 + sol3
    space_avail = len(sol_tot) * B
    loss = (space_avail - np.sum(np.sum(sol_tot))) / space_avail * 100
    losses.append(loss)
    solutions.append(sol_tot)
    if loss < best_loss:
        best_solution = sol_tot
        best_loss = loss
        print(best_loss)

    if (loss < loss_target) and \
        (summarize_results(sol_tot, widths, neckins, B).shape[0] <
        (max_doff_layouts+1)) and \
        (all(pd.DataFrame(sol_tot).replace(0, np.nan, inplace=False)\
        .nunique(axis=1) <= max_unique_products)):
        break
# return sol_tot, loss

1.2740384615384615


In [155]:
Counter(order_remaining)

Counter({225: 15, 215: 7, 241: 6, 175: 1})

In [156]:
def flatten(seq, container=None):
    if container is None:
        container = []

    for s in seq:
        try:
            iter(s)  # check if it's iterable
        except TypeError:
            container.append(s)
        else:
            flatten(s, container)

    return container

In [157]:
# check if find_optimum() creates the right number of jumbos

print(Counter(flatten(sol_tot))-Counter(s))
print((Counter(s)))
print(Counter(flatten(sol_tot)))

Counter()
Counter({241: 460, 175: 109, 215: 75, 158: 32, 225: 22})
Counter({241.0: 460, 175.0: 109, 215.0: 75, 158.0: 32, 225.0: 22})


# Build Layout for Dashboard View

In [158]:
sol

[[175,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  158,
  241,
  241,
  241,
  158],
 [241,
  241,
  241,
  241,
  241,
  215,
  241,
  175,
  241,
  241,
  241,
  175,
  241,
  175,
  215,
  241,
  241,
  175],
 [241,
  215,
  215,
  175,
  215,
  215,
  175,
  158,
  241,
  175,
  175,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241],
 [241,
  241,
  158,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  215,
  158],
 [241,
  241,
  158,
  241,
  241,
  175,
  225,
  241,
  241,
  241,
  241,
  241,
  175,
  241,
  241,
  241,
  215,
  241],
 [175,
  241,
  241,
  241,
  215,
  241,
  175,
  241,
  158,
  175,
  241,
  241,
  241,
  215,
  241,
  241,
  241,
  175,
  215],
 [241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  241,
  175],
 [158,
  175,
  225,
  241,
  175,
  241,
  241,
  241,
  175,
  215,
  241,
  241,
  241,
  215,
  241,

In [21]:
layout_summary(sol, widths, neckins, B)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,Doffs,Loss
Doffs,,,,,,,,,,,,,,,,,,,,,
1,160,160,160,160,160,160,160,160,160,160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1372.0
1,160,160,160,160,160,160,160,160,160,160,...,160.0,160.0,160.0,160.0,160.0,160.0,160.0,160.0,1,60.0
1,195,195,195,195,195,195,195,195,195,195,...,160.0,160.0,160.0,160.0,160.0,NaN,NaN,NaN,1,84.0
2,195,195,195,195,195,195,195,195,195,195,...,195.0,195.0,195.0,NaN,NaN,NaN,NaN,NaN,2,160.0
1,205,205,205,205,205,205,205,205,205,205,...,195.0,195.0,195.0,NaN,NaN,NaN,NaN,NaN,1,0.0
3,205,205,205,205,205,205,205,205,205,205,...,205.0,205.0,160.0,NaN,NaN,NaN,NaN,NaN,3,6.0
1,220,220,220,205,205,205,205,205,205,205,...,205.0,205.0,NaN,NaN,NaN,NaN,NaN,NaN,1,119.0
1,235,235,235,235,235,235,235,235,235,235,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,46.0


# Run Master Schedule, Optimizing Either for On-Time Orders or Fastest Schedule

In [160]:
setup_df = pd.read_excel('../data/200725_WAVA Deckle Optimization Parameters Rev1.xlsx',
                         sheet_name='Deckle Parameters')
speed_df = pd.read_excel('../data/200725_WAVA Deckle Optimization Parameters Rev1.xlsx',
                         sheet_name='Product Parameters')

In [161]:
start_date_time = datetime.datetime(2020,8,1)
start_date_time

datetime.datetime(2020, 8, 1, 0, 0)

In [162]:
extras = pd.DataFrame(np.zeros(len(widths))).T
extras

,0,1,2,3,4
0,0.0,0.0,0.0,0.0,0.0


In [163]:
master2 = make_layout_registrar(sol, widths, neckins)
master2

,154,170,208,218,234,freq
0,1.0,5.0,2.0,1.0,10,1
0,1.0,5.0,1.0,2.0,10,1
0,1.0,2.0,3.0,NaN,12,1
0,1.0,8.0,2.0,NaN,9,1
0,1.0,4.0,3.0,NaN,11,1
0,2.0,1.0,NaN,NaN,15,1
0,NaN,NaN,1.0,NaN,1,1
0,NaN,2.0,2.0,NaN,14,1
0,1.0,1.0,2.0,NaN,14,1
0,1.0,1.0,2.0,NaN,14,1


In [164]:
neckin=neckins
extras = pd.DataFrame(np.zeros(len(widths))).T
master2 = make_layout_registrar(sol, widths, neckin)
master2.columns = master2.columns.str.strip()
DEBUG=True
if DEBUG:
    print(master2.head(10))
extras.columns = master2.columns[:-1]
schedule = []
schedule_with_order_info = []
layout_pattern = 0
old_width = width = None
completed_orders = []
df_filtered['Order Number'] = df_filtered.index + 1


   154  170  208  218  234  freq
0  1.0  5.0  2.0  1.0   10     1
0  1.0  5.0  1.0  2.0   10     1
0  1.0  2.0  3.0  NaN   12     1
0  1.0  8.0  2.0  NaN    9     1
0  1.0  4.0  3.0  NaN   11     1
0  2.0  1.0  NaN  NaN   15     1
0  NaN  NaN  1.0  NaN    1     1
0  NaN  2.0  2.0  NaN   14     1
0  1.0  1.0  2.0  NaN   14     1
0  1.0  1.0  2.0  NaN   14     1


In [165]:

# def optimize_schedule(sol, widths, neckin, df_filtered, L, setup_df, speed_df,
#                   doffs_in_jumbo, start_date_time, 
neckin=neckins
# objective='Time (Knife Changes)'
objective='Late Orders'
DEBUG=True

#####################################################
# Step 1: Create Schedule According to Order Sequence
#####################################################
extras = pd.DataFrame(np.zeros(len(widths))).T
master2 = make_layout_registrar(sol, widths, neckin)
master2.columns = master2.columns.str.strip()
if DEBUG:
    print(master2.head())
extras.columns = master2.columns[:-1]
schedule = []
schedule_with_order_info = []
layout_pattern = 0
old_width = width = None
completed_orders = []
df_filtered['Order Number'] = df_filtered.index + 1

if objective == 'Late Orders':
    #### logic for jumbo rolls should go in first loop
    for row1 in df_filtered.index:
        clear_output(wait=True)
        ### select schedule data
        current_scheduled = df_filtered.iloc[row1][['Total LM Order QTY', 'Width', 'Scheduled Ship Date']]
        current_scheduled['Order Number'] = row1+1
        ship_date = current_scheduled['Scheduled Ship Date']
        doffs = math.ceil(current_scheduled['Total LM Order QTY'] / L) # QTY
        width = str(current_scheduled['Width'])
        if DEBUG:
            print(width)

        ### check if we've entered a new product
        ### no setup changes here, we need to see if the layout has changed
        if width == old_width:
            layout_pattern -= 1

        ### sort by the current scheduled width in master 2
        master2 = master2.sort_values(str(width), ascending=False)
        master2 = master2.reset_index(drop=True)
        if DEBUG:
            print(master2.head())

        ### calc how many new doffs need to be made from inventory
        target_doffs = extras.iloc[0][width]
        if DEBUG:
            print(pd.DataFrame(data=[[doffs, width, target_doffs]],
                               columns=['required doffs', 'width', 'doffs made']))
            print(doffs, width, target_doffs)

        ### only proceed if doffs are available in layout registrar
        ### and not in inventory
        if ((any(master2.loc[master2[width] > 0]['freq'] > 0))
            and (target_doffs <= doffs)):

        ### go through the rows in the master2 registrar and check for
        ### layout patterns that contain our current width

        ### Every row is a new layout unless the outer
        ### loop has incremented, then only maybe is it
        ### a new layout
            for row in master2.index:

                # if none of the width are in the layout then break out
                if np.isnan(master2.iloc[row][width]):
                    break
                layout_pattern += 1

                # looping through available doffs in the layout
                for count in range(master2.iloc[row]['freq'].astype(int)):

                    # add to target doffs
                    target_doffs += master2.iloc[row][width]

                    # add to number of times we've made this layout
                    master2.at[row, 'layout number'] = layout_pattern


                    new_layout_and_scheduled_product = pd.concat([master2.iloc[row],
                                                                  current_scheduled])
                    schedule_with_order_info.append(new_layout_and_scheduled_product)
                    schedule.append(master2.iloc[row])
                    master2.at[row, 'freq'] = master2.at[row, 'freq'] - 1

                    ### Tabluate the other widths that were made on the layout
                    for master_index in master2.iloc[row].index[:-2]:
                        if (math.isnan(master2.iloc[row][master_index]) == False):
                            ### tabulate extras for other widths in the layout
                            extras.iloc[0][master_index] = extras.iloc[0][master_index] +\
                                master2.iloc[row][master_index]
                            
                    if target_doffs >= doffs:

                        ### add to registrar of completed orders.
                        completed_orders.append(current_scheduled['Order Number'])

                        ### now that we've finished this order, we want to see if we've completed any other orders as well


                        break
                ### for exiting nested for loop
                else:
                    continue
                break
        old_width = width
        extra = target_doffs - doffs
        extras.iloc[0][width] = extra
        if DEBUG:
            print(extras)

    sorted_schedule_with_order_info = pd.DataFrame(schedule_with_order_info)
    sorted_schedule_with_order_info = sorted_schedule_with_order_info.reset_index(drop=True)
    sorted_schedule = pd.DataFrame(schedule)
    sorted_schedule = sorted_schedule.reset_index(drop=True)
elif objective == "Time (Knife Changes)":
    # when we sort master, this will be similar to how we sort
    # according to the desirable width in the optimize for late
    # order algorithm. We will however do this for every width

    for width in widths:
        master2 = master2.sort_values(str(width), ascending=False)
        master2 = master2.reset_index(drop=True)

    # now we need to go through the schedule and match layouts
    # with orders

    # go through the layouts
    for row in master2.index:

        # go through doffs in the layout
        for count in range(master2.iloc[row]['freq'].astype(int)):

            # here we will grow our extras dataframe. We will use this
            # to check against orders that are fullfilled.

            ### Tabluate the other widths that were made on the layout
            for master_index in master2.iloc[row].index[:-1]: # no layout number col, -2 in other algorithm

                if (math.isnan(master2.iloc[row][master_index]) == False):

                    ### tabulate extras for other widths in the layout
                    extras.iloc[0][master_index] = extras.iloc[0][master_index] +\
                        master2.iloc[row][master_index]

            # go through the orders
            added_this_roll = []
            if df_filtered.shape[0] > 0:
                for row1 in df_filtered.index:
                    current_scheduled = df_filtered.iloc[row1][['Total LM Order QTY', 'Width',
                                                                     'Scheduled Ship Date',
                                                                     'Order Number']]
                    order_number = current_scheduled['Order Number']
                    ship_date = current_scheduled['Scheduled Ship Date']
                    doffs = math.ceil(current_scheduled['Total LM Order QTY'] / L) # QTY
                    width = str(current_scheduled['Width'])
                    if extras[width][0] >= doffs:

                        completed_orders.append(order_number)
                        extra = extras[width][0] - doffs
                        extras.iloc[0][width] = extra
                        new_layout_and_scheduled_product = pd.concat([master2.iloc[row],
                                                                      current_scheduled])
                        schedule_with_order_info.append(new_layout_and_scheduled_product)
                        added_this_roll.append(order_number)
            # if no order completed just append the layout
            if len(added_this_roll) == 0:
                schedule_with_order_info.append(master2.iloc[row])
            # if two orders completed append both
            elif len(added_this_roll) == 2:
                schedule_with_order_info[-1][-1] =  "{} and {}".format(added_this_roll[0], added_this_roll[1])
                schedule_with_order_info.pop(-2)
            elif len(added_this_roll) > 2:
                print('this is not tested')
                schedule_with_order_info[-1][-1] = str(added_this_roll).split('[')[-1].split(']')[0]
                # schedule_with_order_info.pop(-len(added_this_roll))
            df_filtered = df_filtered[~df_filtered['Order Number'].isin(completed_orders)]
            df_filtered = df_filtered.reset_index(drop=True)
            master2.loc[0, 'freq'] -= 1


    sorted_schedule_with_order_info = pd.DataFrame(schedule_with_order_info)
    sorted_schedule_with_order_info = sorted_schedule_with_order_info.reset_index(drop=True)
    sorted_schedule_with_order_info[['Total LM Order QTY',
   'Width', 'Scheduled Ship Date', 'Order Number']] = \
    sorted_schedule_with_order_info[['Total LM Order QTY',
   'Width', 'Scheduled Ship Date', 'Order Number']].fillna(method='bfill')

234
   154  170  208  218  234  freq  layout number
0  NaN  1.0  NaN  NaN   16     0            4.0
1  2.0  1.0  NaN  NaN   15     0            6.0
2  2.0  NaN  1.0  NaN   15     0            5.0
3  NaN  1.0  1.0  NaN   15     0           15.0
4  1.0  1.0  2.0  NaN   14     0           19.0
   required doffs width  doffs made
0              21   234        20.0
21 234 20.0
   154  170  208  218  234
0  0.0 -1.0 -1.0  0.0 -1.0


In [166]:
completed_orders

[1, 2, 5, 6, 7, 11, 12, 13, 14, 16, 28]

In [167]:
master2

,154,170,208,218,234,freq,layout number
0,NaN,1.0,NaN,NaN,16,0,4.0
1,2.0,1.0,NaN,NaN,15,0,6.0
2,2.0,NaN,1.0,NaN,15,0,5.0
3,NaN,1.0,1.0,NaN,15,0,15.0
4,1.0,1.0,2.0,NaN,14,0,19.0
5,1.0,1.0,2.0,NaN,14,0,20.0
6,NaN,3.0,1.0,NaN,14,0,77.0
7,NaN,2.0,2.0,NaN,14,0,25.0
8,NaN,3.0,2.0,NaN,13,0,39.0
9,NaN,2.0,1.0,2.0,13,0,76.0


In [168]:
extras

,154,170,208,218,234
0,0.0,-1.0,-1.0,0.0,-1.0


In [169]:
sorted_schedule_with_order_info

,154,170,208,218,234,freq,layout number,Total LM Order QTY,Width,Scheduled Ship Date,Order Number
0,NaN,2.0,5.0,NaN,11.0,1.0,1.0,3.698225e+05,208,2020-07-21,1
1,1.0,8.0,2.0,NaN,9.0,1.0,2.0,2.262443e+06,170,2020-07-21,2
2,1.0,6.0,1.0,1.0,10.0,1.0,3.0,2.262443e+06,170,2020-07-21,2
3,1.0,5.0,1.0,2.0,10.0,1.0,4.0,2.262443e+06,170,2020-07-21,2
4,1.0,5.0,2.0,1.0,10.0,1.0,5.0,2.262443e+06,170,2020-07-21,2
5,NaN,1.0,NaN,NaN,16.0,1.0,4.0,2.136752e+06,234,2020-07-22,5
6,2.0,NaN,1.0,NaN,15.0,1.0,5.0,2.136752e+06,234,2020-07-22,6
7,2.0,1.0,NaN,NaN,15.0,1.0,6.0,2.136752e+06,234,2020-07-22,6
8,NaN,4.0,2.0,NaN,12.0,1.0,11.0,2.941176e+06,170,2020-07-22,7
9,3.0,4.0,NaN,3.0,9.0,1.0,12.0,2.941176e+06,170,2020-07-22,7


In [170]:
# check that schedule is creating correct number of widths
print(sorted_schedule_with_order_info.iloc[:,:len(widths)].sum())
print(Counter(flatten(sol_tot))-Counter(s))
print((Counter(s)))
print(Counter(flatten(sol_tot)))

154     32.0
170    109.0
208     75.0
218     22.0
234    460.0
dtype: float64
Counter()
Counter({241: 460, 175: 109, 215: 75, 158: 32, 225: 22})
Counter({241.0: 460, 175.0: 109, 215.0: 75, 158.0: 32, 225.0: 22})


In [171]:
extras

,154,170,208,218,234
0,0.0,-1.0,-1.0,0.0,-1.0


In [172]:
# check that schedule is creating correct number of widths
print(sorted_schedule_with_order_info.iloc[:,:len(widths)].sum())
print(Counter(flatten(sol_tot))-Counter(s))
print((Counter(s)))
print(Counter(flatten(sol_tot)))

154     32.0
170    109.0
208     75.0
218     22.0
234    460.0
dtype: float64
Counter()
Counter({241: 460, 175: 109, 215: 75, 158: 32, 225: 22})
Counter({241.0: 460, 175.0: 109, 215.0: 75, 158.0: 32, 225.0: 22})


In [173]:
#####################################################
# Step 2: Add Times Based on Rates/Changeovers
#####################################################
### requires setup_df, speed_df

# slitter_speed = speed_df.loc[(speed_df['Customer Name'] == customer) &
#          (speed_df['Description'].str.contains(technology)) &
#          (speed_df['Description'].str.contains(color)), 'Slittter Speed (m/min)'].reset_index(drop=True)[0]
slitter_speed = 1300

# add column 'completion date/time'
# for row in schedule, calculate the completion time
sorted_schedule_with_order_info['Completion Date'] = None

### choose starte date/time
# start_date_time = datetime.datetime(2020, 8, 12)
if DEBUG:
    print("start time: {}".format(start_date_time))

### set changeover times
jumbo_change = setup_df.loc[setup_df['Slitter Set-up'] ==
                            'Jumbo roll only']['Time (minutes)'].values[0]
jumbo_change = datetime.timedelta(minutes=jumbo_change)
if DEBUG:
    print("jumbo change: {}".format(jumbo_change))
jumbo_and_knife_change = setup_df.loc[setup_df['Slitter Set-up'] ==
                                      'AMB & Arium']['Time (minutes)'].values[0]
jumbo_and_knife_change = datetime.timedelta(minutes=jumbo_and_knife_change)
if DEBUG:
    print("jumbo and knife change: {}".format(jumbo_and_knife_change))

### print slitter speed and doff length
if DEBUG:
    print("slitter speed (m/min): {}".format(slitter_speed))
    print("doff length (m): {}".format(L))
prior_completion_date_time = start_date_time
match = '\d\d\d'
layout_columns = [i for i in sorted_schedule_with_order_info.columns if re.match(match, i)]

### zero-out the 'prior layout' so algorithm knows jumbo + knife change
prior_layout = sorted_schedule_with_order_info.iloc[0][layout_columns]
for col in prior_layout.index:
    prior_layout[col] = 0
layout_number = 0

schedule_with_change_over = pd.DataFrame()
### transition times are for loading the current jumbo
for row in sorted_schedule_with_order_info.index:

    layout = sorted_schedule_with_order_info.iloc[row][layout_columns]
    if all(prior_layout.fillna(0) == layout.fillna(0)):
        transition_time = jumbo_change
    else:
        transition_time = jumbo_and_knife_change
        layout_number += 1
    if DEBUG:
        print("transition time: {}".format(transition_time))
    run_time = L / slitter_speed
    run_time = datetime.timedelta(minutes=run_time)
    if DEBUG:
        print("run time: {}".format(run_time))
    completion_time = transition_time + run_time
    completion_date_time = prior_completion_date_time + completion_time
    if DEBUG:
        print("completion date/time: {}".format(completion_date_time))

    ### change main df
    sorted_schedule_with_order_info['Completion Date'][row] = completion_date_time
    # sorted_schedule_with_order_info['layout number'][row] = layout_number

    ### with changeover rows
    current_changeover = pd.DataFrame(sorted_schedule_with_order_info.columns)
    current_changeover = current_changeover.set_index(0)
    current_changeover = current_changeover.T
    current_changeover = current_changeover.append({'Order Number': 'Changeover'}, ignore_index=True)
    current_changeover['Completion Date'] = prior_completion_date_time + transition_time
    current_order = pd.DataFrame(sorted_schedule_with_order_info.iloc[row]).T
    current_combined = current_changeover.append(current_order, sort=False, ignore_index=True)

    schedule_with_change_over = pd.concat([schedule_with_change_over, current_combined])

    prior_completion_date_time = completion_date_time
    prior_layout = layout
    if not DEBUG:
        clear_output(wait=True)

schedule_with_change_over = schedule_with_change_over.reset_index(drop=True)

#####################################################
# Step 3: Create Summary With Changeover Rows
#####################################################

master_schedule = pd.DataFrame()
unformatted_schedule = pd.DataFrame()
for index in schedule_with_change_over.index[1::2]: # pass through every jumbo
    order_number = schedule_with_change_over.iloc[index]['Order Number']

    ### these are columns specific to width layouts
    match = '\d\d\d'
    layout_columns = [i for i in schedule_with_change_over.columns if re.match(match, i)]


    deckle_layout =  deckle_order = pd.DataFrame(schedule_with_change_over.iloc[index]).T

    ### these are columns specific to order details
    order_columns = [i for i in deckle_order.columns if (i not in layout_columns)
                     & (i not in ['freq', 'layout number'])]

    ### for order we grab the last row since this is the real 'completion date'
    order = deckle_order[order_columns].reset_index(drop=True).iloc[-1].dropna()

    ### remove widths that are in the deckle opt. but not in the layout
    formula = deckle_layout[layout_columns].reset_index(drop=True).iloc[0].dropna()

    ### number of doffs for the given layout/order
    doffs = doffs_in_jumbo
    read_out = ''
    for i in range(formula.shape[0]-1): #for unique prods
        read_out = read_out + ("{}x{} + ".format(formula.index[i],
                                                formula.iloc[i].astype(int)))
    read_out = read_out + ("{}x{}".format(formula.index[-1],
                                        formula.iloc[-1].astype(int))) #make last string w/o +
    current = pd.DataFrame([read_out, int(doffs), order]).T
    order = pd.DataFrame(order).T
    order = order.reset_index(drop=True)
    current = pd.DataFrame([read_out, int(doffs)]).T
    current.columns = ['Formula', 'Doffs']
    current = current.join(order)

    ### add changeover info
    current_changeover = pd.DataFrame(current.columns)
    current_changeover = current_changeover.set_index(0)
    current_changeover = current_changeover.T
    current_changeover = current_changeover.append({'Order Number': 'Changeover'}, ignore_index=True)
    current_changeover['Completion Date'] = schedule_with_change_over.iloc[index-1]['Completion Date']
    current_combined = current_changeover.append(current, sort=False, ignore_index=True)
    current_combined

    master_schedule = pd.concat([master_schedule, current_combined], sort=False)
    master_schedule = master_schedule.reset_index(drop=True)

start time: 2020-08-01 00:00:00
jumbo change: 0:15:00
jumbo and knife change: 0:50:00
slitter speed (m/min): 1300
doff length (m): 102000
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-01 02:08:27.692308
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-01 04:16:55.384616
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-01 06:25:23.076924
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-01 08:33:50.769232
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-01 10:42:18.461540
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-01 12:50:46.153848
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-01 14:59:13.846156
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-01 17:07:41.538464
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time

<ipython-input-173-6f285c35f796>:68: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-02 17:39:13.846160
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-02 19:47:41.538468
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-02 21:56:09.230776
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-03 00:04:36.923084
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-03 02:13:04.615392
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-03 04:21:32.307700
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-03 06:30:00.000008
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-03 08:38:27.692316
transition time: 0:50:00
run time: 1:18:27.692308
completion date/time: 2020-08-03 10:46:55.384624
transition time: 0:15:00
run time: 1:18:27.692308
completion date/time: 2020-08-03 12:20:23.076932
transition

In [174]:
extras
out = "Inventory created: "
for col in extras:
    if extras[col][0] != 0:
        out += "{} x {:.0f}; ".format(col, extras[col][0])
out = out[:-2]
print(out)

Inventory created: 170 x -1; 208 x -1; 234 x -1


In [175]:
extras

,154,170,208,218,234
0,0.0,-1.0,-1.0,0.0,-1.0


In [176]:
master_knife, extras = optimize_schedule(sol, widths, neckins, df_filtered, L, setup_df, 
                           speed_df, doffs_in_jumbo, start_date_time, 'Time (Knife Changes)')
deckle_time = master_knife.iloc[-1]['Completion Date'] - master_knife.iloc[0]['Completion Date']
print("deckle time, knife changes objective: {}".format(str(deckle_time).split('.')[0]))

deckle time, knife changes objective: 3 days 08:55:00


In [177]:
extras

,154,170,208,218,234
0,0.0,28.0,23.0,0.0,20.0


In [178]:
master_late, extras = optimize_schedule(sol, widths, neckins, df_filtered, L, setup_df, 
                           speed_df, doffs_in_jumbo, start_date_time, 'Late Orders')
deckle_time = master_late.iloc[-1]['Completion Date'] - master_late.iloc[0]['Completion Date']
print("deckle time, late orders objective: {}".format(str(deckle_time).split('.')[0]))

deckle time, late orders objective: 3 days 08:20:00


In [179]:
extras

,154,170,208,218,234
0,0.0,-1.0,-1.0,0.0,-1.0
